<h3>Importing the Libraries</h3>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

import lightning as L
from torch.utils.data import TensorDataset, DataLoader

/home/sum/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


<h3>Creating the LSTM Class</h3>

In [2]:
class LSTMByHand(L.LightningModule):
    def __init__(self):
        #Create and initialize Weight and Bias tensors

        super().__init__()
        mean = torch.tensor(0.)
        std = torch.tensor(1.)

        self.wlr1 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        self.wlr2 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        self.blr1 = nn.Parameter(torch.tensor(0.), requires_grad = True)

        self.wpr1 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        self.wpr2 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        self.bpr1 = nn.Parameter(torch.tensor(0.), requires_grad = True)

        self.wp1 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        self.wp2 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        self.bp1 = nn.Parameter(torch.tensor(0.), requires_grad = True)

        self.wo1 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        self.wo2 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        self.bo1 = nn.Parameter(torch.tensor(0.), requires_grad = True)        


    def lstm_unit(self, input_value, long_memory, short_memory):
        #Do the LSTM math

        long_remember_precent = torch.sigmoid((short_memory * self.wlr1) +
                                              (input_value * self.wlr2) +
                                              self.blr1)
        
        potential_remember_precent = torch.sigmoid((short_memory * self.wpr1) +
                                              (input_value * self.wpr2) +
                                              self.bpr1)
        
        potential_memory = torch.tanh((short_memory * self.wp1) +
                                              (input_value * self.wp2) +
                                              self.bp1)
        
        updated_long_memory = ((long_memory * long_remember_precent) +
                               (potential_remember_precent * potential_memory))
        
        output_percent = torch.sigmoid((short_memory * self.wo1) +
                                       (input_value * self.wo2) +
                                       self.bo1)
        
        updated_short_memory = torch.tanh(updated_long_memory) * output_percent

        return [updated_long_memory, updated_short_memory]


    def forward(self, input):
        #Make a forward pass through the unrolled LSTN
        
        long_memory =  0
        short_memory = 0
        day1 = input[0]
        day2 = input[1]
        day3 = input[2]
        day4 = input[3]

        long_memory, short_memory = self.lstm_unit(day1, long_memory, short_memory)
        long_memory, short_memory = self.lstm_unit(day2, long_memory, short_memory)
        long_memory, short_memory = self.lstm_unit(day3, long_memory, short_memory)
        long_memory, short_memory = self.lstm_unit(day4, long_memory, short_memory)

        return short_memory


    def configure_optimizers(self):
        #Configure the Adam optimizer
        
        return Adam(self.parameters())


    def training_step(self, batch, batch_idx):
        #Calculate loss and log the training process
        
        input_i, label_i = batch
        output_i = self.forward(input_i[0])
        loss = (output_i - label_i) ** 2

        self.log("train_loss", loss)

        if label_i == 0:
            self.log("out_0", output_i)
        else:
            self.log("out_1", output_i)

        return loss

<h3>Making Predictions with the Initial Random Values

In [21]:
model = LSTMByHand()

print("Now let's compare the observed and predicted values:")
print("\nCompany A: Observed = 0, Predicted =",
      model(torch.tensor([0., 0.5, 0.25, 1.])).detach())
print("Company B: Observed = 1, Predicted =",
      model(torch.tensor([1., 0.5, 0.25, 1.])).detach())

Now let's compare the observed and predicted values:

Company A: Observed = 0, Predicted = tensor(0.5078)
Company B: Observed = 1, Predicted = tensor(0.6223)


<h3>Training the Model</h3>

In [26]:
inputs = torch.tensor([[0., 0.5, 0.25, 1.], [1., 0.5, 0.25, 1.]])
labels = torch.tensor([0., 1.])     #Expected (actual) outputs for Companies A & B

dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

trainer = L.Trainer(max_epochs = 2000)
trainer.fit(model, train_dataloaders = dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Missing logger folder: /home/sum/Study/Statquest_Codes/lightning_logs
2024-07-03 19:25:28.440990: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-03 19:25:28.597300: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-03 19:25:28.676742: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-03 19:25:28.677655: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to 

Epoch 1999: 100%|██████████| 2/2 [00:00<00:00, 111.76it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=2000` reached.


Epoch 1999: 100%|██████████| 2/2 [00:00<00:00, 83.03it/s, v_num=0] 


<h3>Making Predictions After Training the Model</h3>

In [27]:
print("Now let's compare the observed and predicted values:")
print("\nCompany A: Observed = 0, Predicted =",
      model(torch.tensor([0., 0.5, 0.25, 1.])).detach())
print("Company B: Observed = 1, Predicted =",
      model(torch.tensor([1., 0.5, 0.25, 1.])).detach())

Now let's compare the observed and predicted values:

Company A: Observed = 0, Predicted = tensor(0.0003)
Company B: Observed = 1, Predicted = tensor(0.9706)
